<a href="https://colab.research.google.com/github/FuriouStyles/DS-Unit-2-Kaggle-Challenge/blob/master/module3/Stephen_P_LS_DS_223_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [x] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


You won't be able to just copy from the lesson notebook to this assignment.

- Because the lesson was ***regression***, but the assignment is ***classification.***
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [0]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [0]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(train, test_size=0.2)

In [0]:
# I'm going to do the following for this iteration: convert my date column to a timestamp, add haversine distance to nearest pump and 
# binary rainy season when checked features, and use LabelEncoder() to convert everything into format that works with WOEEncoder()

import numpy as np
from math import radians, cos, sin, asin, sqrt


def wrangle(x):
  x = x.copy()
  x['latitude'] = x['latitude'].replace(-2e-08, 0)
  col_with_zeros = ['latitude', 'longitude']
  for col in col_with_zeros:
    x[col] = x[col].replace(0, np.nan)
  x = x.drop(columns=['quantity'])
  def single_pt_haversine(lat, lng, degrees=True):
    r = 6371
    if degrees:
      lat, lng = map(radians, [lat, lng])
    a = sin(lat/2)**2 + cos(lat) * sin(lng/2)**2
    d = 2*r*asin(sqrt(a))
    return d
  x['haversine_distance'] = [single_pt_haversine(LAT, LONG) for LAT, LONG in zip(x['latitude'], x['longitude'])]
  pop_per_basin = x.groupby(train['basin'])['population'].sum()
  pump_per_basin = x.groupby(train['basin'])['id'].count()
  basin_pop_pump_ratio = pump_per_basin/pop_per_basin
  basin_pop_pump_ratio.to_dict()
  x['basin_pop_pump_ratio'] = x['basin'].map(basin_pop_pump_ratio)
  x['timestamp'] = pd.to_datetime(x['date_recorded'])
  def rainy_season(y):
    if (y.month == 3) or (y.month == 4) or (y.month == 5):
      return True
    else:
      return False
  x['rainy_season'] = [rainy_season(i) for i in x['timestamp']]
  x = x.drop(columns=['timestamp'])
  x['closest_pump'] = [np.abs(x['haversine_distance'] - i).argmin() for i in x['haversine_distance']]
  return x


In [5]:
train = wrangle(train)
val = wrangle(val)
test = wrangle(test)

train.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: FutureWarning: 
The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.


,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group,haversine_distance,basin_pop_pump_ratio,rainy_season,closest_pump
18405,16175,0.0,2013-03-23,Kkkt Church,1309,KKKT CHURCH,37.910991,-4.414261,Vibandani,0,Pangani,Gavao,Kilimanjaro,3,3,Same,Hedaru,210,True,GeoData Consultants Ltd,Water authority,Nguruta water supply,True,2002,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe,non functional,4239.717761,0.005652,True,18405.0
14252,31276,20.0,2013-01-20,Sabodo,12,Sabodo,39.577335,-10.150225,Kwa Binti Ndembo,0,Ruvuma / Southern Coast,Nakayaya,Lindi,80,23,Lindi Rural,Kiwalala,1,True,GeoData Consultants Ltd,VWC,Sabodo Borehole Scheme,False,2011,india mark ii,india mark ii,handpump,vwc,user-group,pay per bucket,per bucket,salty,salty,insufficient,machine dbh,borehole,groundwater,hand pump,hand pump,functional,4520.083183,0.003641,False,14252.0
3064,36204,0.0,2013-01-23,Kigoma Municipal,807,Kigoma municipal,29.664953,-4.858918,Kwa Makanyaga,0,Lake Tanganyika,Mgeo,Kigoma,16,4,Kigoma Urban,Buhanda businde,0,True,GeoData Consultants Ltd,VWC,NaN,True,2009,nira/tanira,nira/tanira,handpump,vwc,user-group,never pay,never pay,soft,good,dry,shallow well,shallow well,groundwater,hand pump,hand pump,non functional,3338.569656,0.003824,False,3064.0
52061,42410,500.0,2011-02-28,Tz Japan,1662,DWE,34.915908,-8.969763,Kwa Josia Kadwame,0,Rufiji,Igongolo B,Iringa,11,4,Njombe,Igongolo,40,True,GeoData Consultants Ltd,VWC,Igongolo gravity water sche,False,2006,gravity,gravity,gravity,vwc,user-group,pay monthly,monthly,soft,good,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional,3992.729436,0.005853,False,52061.0
4196,69171,100.0,2011-03-16,African Development Bank,96,DAWASCO,38.989375,-6.774482,Felesta Felesiana,0,Wami / Ruvu,Tangini,Pwani,6,2,Kibaha,Mailimoja,1,False,GeoData Consultants Ltd,Water authority,NaN,True,2010,mono,mono,motorpump,private operator,commercial,pay per bucket,per bucket,soft,good,insufficient,river,river/lake,surface,communal standpipe,communal standpipe,functional,4390.082421,0.003935,True,4196.0


In [8]:
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
from scipy.stats import randint, uniform
from sklearn.preprocessing import LabelEncoder

target = 'status_group'
features = train.drop(columns=[target, 'id']).columns.tolist()

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_val = le.fit_transform(y_val)


pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier()
)

param_distributions = {
    'simpleimputer__strategy': ['mean', 'median'],
    'randomforestclassifier__n_estimators': randint(50, 500),
    'randomforestclassifier__max_depth': [5, 10, 15, 20, None],
    'randomforestclassifier__max_features': uniform(0, 1),
    'randomforestclassifier__min_samples_leaf': randint(1, 10)
}

search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distributions,
    n_iter=100,
    cv=4,
    scoring='neg_mean_absolute_error',
    verbose=10,
    return_train_score=True,
    n_jobs=-1
)

search.fit(X_train, y_train)

Fitting 4 folds for each of 100 candidates, totalling 400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  7.8min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed: 11.3min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed: 15.2min
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed: 16.6min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 21.3min
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 24.1min
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  57 tasks      | elapsed: 40.2min
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed: 45.0min
[Parallel(n_jobs=-1)]: Done 

RandomizedSearchCV(cv=4, error_score='raise-deprecating',
                   estimator=Pipeline(memory=None,
                                      steps=[('ordinalencoder',
                                              OrdinalEncoder(cols=None,
                                                             drop_invariant=False,
                                                             handle_missing='value',
                                                             handle_unknown='value',
                                                             mapping=None,
                                                             return_df=True,
                                                             verbose=0)),
                                             ('simpleimputer',
                                              SimpleImputer(add_indicator=False,
                                                            copy=True,
                                                            f

In [12]:
print(f'Best hyperparamters: {search.best_params_}')
print(f'Best estimator: {search.best_estimator_}') 

Best hyperparamters: {'randomforestclassifier__max_depth': 20, 'randomforestclassifier__max_features': 0.29837257077811674, 'randomforestclassifier__min_samples_leaf': 3, 'randomforestclassifier__n_estimators': 417, 'simpleimputer__strategy': 'median'}
Best estimator: Pipeline(memory=None,
         steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['date_recorded', 'funder', 'installer',
                                      'wpt_name', 'basin', 'subvillage',
                                      'region', 'lga', 'ward', 'public_meeting',
                                      'recorded_by', 'scheme_management',
                                      'scheme_name', 'permit',
                                      'extraction_type',
                                      'extraction_type_group',
                                      'extraction_type_class', 'management',
                                      'management_group', 'payment',
                                      '

In [13]:
tuned_pipeline = search.best_estimator_

#y_val_pred = tuned_pipeline.predict(X_val)
print(f'Validation Accuracy: {tuned_pipeline.score(X_val, y_val)}')

Validation Accuracy: 0.8118686868686869
